# Primary Health Care Accessibility in Ontario

In [1]:
#importing all necessary libraries for data analysis and visualization

import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt
import geopandas

## 1. Data Inspection and Cleaning 

In [4]:
#source data downloaded from:https://data.ontario.ca/dataset/family-health-team-fht-locations

# data text file loaded in pandas and inspected to identify significant attribute for our analysis 
url = 'Ministry_of_Health_Service_Provider_Locations.csv'
location_data = pd.read_csv(url, sep = ',',  encoding='latin-1')
location_data.info()
location_data.drop(['FRENCH_NAME', 'FRENCH_NAME_ALT', 'ENGLISH_NAME_ALT'], axis = 1)
location_data['SERVICE_TYPE'].unique()

#Service Type re-cateegorized to identify group of Primary Care Provider services:
location_data['NEW_SERVICE_TYPE'] = location_data['SERVICE_TYPE']
location_data.loc[location_data['SERVICE_TYPE'].str.contains('Family Health Team|Nurse Practitioner-Led Clinic|Indigenous Primary Health Care Organization|Integrated Community Health Services Centre|Nursing Station'), 'NEW_SERVICE_TYPE'] = 'Primary Care Provider'
location_data['NEW_SERVICE_TYPE'].unique()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11749 entries, 0 to 11748
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ï»¿X                        11749 non-null  float64
 1   Y                           11749 non-null  float64
 2   OGF_ID                      11749 non-null  int64  
 3   MOH_SERVICE_PROVIDER_IDENT  11749 non-null  object 
 4   SERVICE_TYPE                11749 non-null  object 
 5   SERVICE_TYPE_DETAIL         9976 non-null   object 
 6   ENGLISH_NAME                11749 non-null  object 
 7   FRENCH_NAME                 88 non-null     object 
 8   ENGLISH_NAME_ALT            916 non-null    object 
 9   FRENCH_NAME_ALT             4 non-null      object 
 10  ADDRESS_LINE_1              11749 non-null  object 
 11  ADDRESS_LINE_2              2813 non-null   object 
 12  ADDRESS_DESCRIPTOR          2173 non-null   object 
 13  COMMUNITY                   117

array(['Licensed Lab and Specimen Collection Centre Locations',
       'Licenced Retirement Home', 'Pharmacy', 'Long-Term Care Home',
       'Community Support Services', 'Primary Care Provider',
       'Mental Health and Addiction Organization',
       'HIV/AIDS and Hep C Programs', 'Seniors Active Living Centre',
       'Hospital', "Children's Treatment Centre", 'Midwifery Clinic',
       'Public Health Unit Office Locations', 'Community Health Centre'],
      dtype=object)

In [6]:
# Mainly : primary care provider density: count/km2 for regional can be calculated 
# Q1. Rural areas vs highly populated cities, type of primary care provider services offered 
# Q2. primary care provider: km2/population identification to understand the accessibility by the population around the clinics
# A. Is the primary care provider serving a population size that is an outlier from provincial standards measured by the distance from any other provider 
# B. Is the primary care provider, nearest provider to a population centre which will be considered inaccessable; i.e. greater 30 min commute time especially for small towns perhaps in northern ontario 


In [8]:
#source data : https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=9810001901
census_data = pd.read_csv('population count per fsc.csv', sep = ',', skiprows = 11, skipfooter = 8, names = ['forward sortation area', 'population_2021', 'Total private dwelling_2021', 'private dwelling by usual resident_2021'], engine =  'python')
census_data.info()
location_data.info()
location_data = location_data.rename(columns = {'ï»¿X' :'X'})


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1646 entries, 0 to 1645
Data columns (total 4 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   forward sortation area                   1646 non-null   object
 1   population_2021                          1646 non-null   object
 2   Total private dwelling_2021              1646 non-null   object
 3   private dwelling by usual resident_2021  1646 non-null   object
dtypes: object(4)
memory usage: 51.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11749 entries, 0 to 11748
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ï»¿X                        11749 non-null  float64
 1   Y                           11749 non-null  float64
 2   OGF_ID                      11749 non-null  int64  
 3   MOH_SERVICE_PROVIDER_IDENT  11749 non-nu

## 2. Data Preparation

In [12]:
#create FSC column and match the FSC values from population dataset to combined corresponding population that the clinics located in. 
location_data['FSC'] = location_data['POSTAL_CODE'].str[:3]
population_fsc = census_data.drop(['Total private dwelling_2021', 'private dwelling by usual resident_2021'], axis =1)
population_fsc.set_index('forward sortation area', inplace = True)
population_fsc_dict = population_fsc.to_dict()

location_data['matched_FSC'] = location_data['FSC'].apply(lambda x: population_fsc_dict['population_2021'].get(x))
location_data['population'] = location_data['matched_FSC'].str.split(',').str.join('')
location_data['population'] = pd.to_numeric(location_data['population'])


location_data = location_data.drop(['OGF_ID', 'MOH_SERVICE_PROVIDER_IDENT','FRENCH_NAME','FRENCH_NAME_ALT', 'ENGLISH_NAME_ALT', 'matched_FSC'], axis = 1)
primary_care_provider = location_data.loc[(location_data['NEW_SERVICE_TYPE'] == 'Primary Care Provider')]


population      
          mean count
FSC                 
K0A   111626.0     7
K0B    21020.0     1
K0C    52838.0     2
K0E    39649.0     2
K0H    47286.0     2
..         ...   ...
P7J     4859.0     1
P8N     9619.0     1
P8T     5541.0     1
P9A    10079.0     2
P9N    14088.0     3

[360 rows x 2 columns]

In [30]:
fsc_count_df =  primary_care_provider.groupby(['FSC']).agg({'SERVICE_TYPE_DETAIL': 'count'})
fsc_count_dic = fsc_count_df.to_dict()

primary_care_provider['pop_count'] = primary_care_provider['FSC'].apply(lambda x: fsc_count_dic['SERVICE_TYPE_DETAIL'].get(x))

,X,Y,SERVICE_TYPE,SERVICE_TYPE_DETAIL,ENGLISH_NAME,ADDRESS_LINE_1,ADDRESS_LINE_2,ADDRESS_DESCRIPTOR,COMMUNITY,POSTAL_CODE,GEOMETRY_UPDATE_DATETIME,EFFECTIVE_DATETIME,SYSTEM_DATETIME,OBJECTID,NEW_SERVICE_TYPE,FSC,population,pop_count
7,-79.787369,47.062699,Family Health Team,Site,Temagami Medical Centre and Family Health Team,17 O'Connor Drive,Postal Office Box 98,NaN,Temagami,P0H2H0,2024/06/25 13:38:05+00,2024/06/25 11:36:47+00,2024/06/25 21:54:59+00,2067313,Primary Care Provider,P0H,31909.0,4
22,-81.314160,48.485990,Integrated Community Health Services Centre,Obs/Gyn Ultrasound,MyHealth Centre,707 Ross Avenue East,NaN,NaN,Timmins,P4N8R1,2024/06/25 13:36:07+00,2024/06/25 11:45:48+00,2024/06/25 21:54:59+00,2067553,Primary Care Provider,P4N,24098.0,5
55,-79.403372,43.657331,Integrated Community Health Services Centre,"Bone Mineral DXA, Mammography, Radiography, Ge...",Kensington Diagnostic Imaging Centre,340 College Street,NaN,NaN,Toronto,M5T3A9,2024/06/25 13:37:26+00,2024/06/25 11:51:23+00,2024/06/25 21:54:59+00,2068081,Primary Care Provider,M5T,17903.0,8
57,-79.254245,43.774259,Integrated Community Health Services Centre,"Nuclear Medicine - In Vivo - General, Nuclear ...",KMH Cardiology Centres Inc.,55 Town Centre Court,NaN,NaN,Scarborough,M1P4X4,2024/06/25 13:36:09+00,2024/06/25 11:45:34+00,2024/06/25 21:54:59+00,2068113,Primary Care Provider,M1P,45170.0,11
64,-79.288799,43.803133,Integrated Community Health Services Centre,Vascular Ultrasound,Lighthouse Medical Imaging - Finch,4168 Finch Avenue East,NaN,NaN,Scarborough,M1S5H6,2024/06/25 13:34:12+00,2024/06/25 11:46:42+00,2024/06/25 21:54:59+00,2068225,Primary Care Provider,M1S,37663.0,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11737,-84.548399,48.346711,Nursing Station,Site,Dubreuilville Nursing Clinic,851 Street . Joseph Street,Postal Office Box 250,Practice Location,Dubreuilville,P0S1B0,2024/07/30 15:10:18+00,2024/07/30 14:58:02+00,2024/07/30 21:49:59+00,2257297,Primary Care Provider,P0S,10170.0,2
11738,-86.707652,50.178995,Nursing Station,Site,Nakina Medical Clinic,204A Thorton Street,Postal Office Box 135,Practice Location,Nakina,P0T2H0,2024/07/30 15:10:18+00,2024/07/30 14:36:25+00,2024/07/30 21:49:59+00,2257313,Primary Care Provider,P0T,32000.0,10
11739,-94.071854,49.396903,Nursing Station,Site,Sioux Narrows-Nestor Falls Nursing Station,10 Fickas Road,NaN,Practice Location,Sioux Narrows,P0X1N0,2024/07/30 15:10:18+00,2024/07/30 11:10:31+00,2024/07/30 21:49:59+00,2257329,Primary Care Provider,P0X,8410.0,1
11741,-82.035869,49.307910,Nursing Station,Site,Fauquier-Strickland Health Center,27 Grzela Road,Postal Office Box 40,Practice Location,Fauquier,P0L1G0,2024/07/30 15:10:18+00,2024/07/30 14:33:53+00,2024/07/30 21:49:59+00,2257361,Primary Care Provider,P0L,22859.0,4


In [ ]:
primary_care_provider['pop_per_clinic'] = primary_care_provider['population']/primary_care_provider['pop_count']

,X,Y,SERVICE_TYPE,SERVICE_TYPE_DETAIL,ENGLISH_NAME,ADDRESS_LINE_1,ADDRESS_LINE_2,ADDRESS_DESCRIPTOR,COMMUNITY,POSTAL_CODE,GEOMETRY_UPDATE_DATETIME,EFFECTIVE_DATETIME,SYSTEM_DATETIME,OBJECTID,NEW_SERVICE_TYPE,FSC,population,pop_count,pop_per_clinic
7,-79.787369,47.062699,Family Health Team,Site,Temagami Medical Centre and Family Health Team,17 O'Connor Drive,Postal Office Box 98,NaN,Temagami,P0H2H0,2024/06/25 13:38:05+00,2024/06/25 11:36:47+00,2024/06/25 21:54:59+00,2067313,Primary Care Provider,P0H,31909.0,4,7977.250000
22,-81.314160,48.485990,Integrated Community Health Services Centre,Obs/Gyn Ultrasound,MyHealth Centre,707 Ross Avenue East,NaN,NaN,Timmins,P4N8R1,2024/06/25 13:36:07+00,2024/06/25 11:45:48+00,2024/06/25 21:54:59+00,2067553,Primary Care Provider,P4N,24098.0,5,4819.600000
55,-79.403372,43.657331,Integrated Community Health Services Centre,"Bone Mineral DXA, Mammography, Radiography, Ge...",Kensington Diagnostic Imaging Centre,340 College Street,NaN,NaN,Toronto,M5T3A9,2024/06/25 13:37:26+00,2024/06/25 11:51:23+00,2024/06/25 21:54:59+00,2068081,Primary Care Provider,M5T,17903.0,8,2237.875000
57,-79.254245,43.774259,Integrated Community Health Services Centre,"Nuclear Medicine - In Vivo - General, Nuclear ...",KMH Cardiology Centres Inc.,55 Town Centre Court,NaN,NaN,Scarborough,M1P4X4,2024/06/25 13:36:09+00,2024/06/25 11:45:34+00,2024/06/25 21:54:59+00,2068113,Primary Care Provider,M1P,45170.0,11,4106.363636
64,-79.288799,43.803133,Integrated Community Health Services Centre,Vascular Ultrasound,Lighthouse Medical Imaging - Finch,4168 Finch Avenue East,NaN,NaN,Scarborough,M1S5H6,2024/06/25 13:34:12+00,2024/06/25 11:46:42+00,2024/06/25 21:54:59+00,2068225,Primary Care Provider,M1S,37663.0,14,2690.214286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11737,-84.548399,48.346711,Nursing Station,Site,Dubreuilville Nursing Clinic,851 Street . Joseph Street,Postal Office Box 250,Practice Location,Dubreuilville,P0S1B0,2024/07/30 15:10:18+00,2024/07/30 14:58:02+00,2024/07/30 21:49:59+00,2257297,Primary Care Provider,P0S,10170.0,2,5085.000000
11738,-86.707652,50.178995,Nursing Station,Site,Nakina Medical Clinic,204A Thorton Street,Postal Office Box 135,Practice Location,Nakina,P0T2H0,2024/07/30 15:10:18+00,2024/07/30 14:36:25+00,2024/07/30 21:49:59+00,2257313,Primary Care Provider,P0T,32000.0,10,3200.000000
11739,-94.071854,49.396903,Nursing Station,Site,Sioux Narrows-Nestor Falls Nursing Station,10 Fickas Road,NaN,Practice Location,Sioux Narrows,P0X1N0,2024/07/30 15:10:18+00,2024/07/30 11:10:31+00,2024/07/30 21:49:59+00,2257329,Primary Care Provider,P0X,8410.0,1,8410.000000
11741,-82.035869,49.307910,Nursing Station,Site,Fauquier-Strickland Health Center,27 Grzela Road,Postal Office Box 40,Practice Location,Fauquier,P0L1G0,2024/07/30 15:10:18+00,2024/07/30 14:33:53+00,2024/07/30 21:49:59+00,2257361,Primary Care Provider,P0L,22859.0,4,5714.750000
